In [3]:
import torch
from transformers import*
from torch import nn

In [65]:
bert_path = 'F:\weibo_hot\Chinese_bert\data'
bert_size = 768
data_path = 'F:/weibo_hot/data'
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(bert_path)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(bert_size, 3)
        
    def forward(self, x):
        context = x[0]  # 输入的句子
        mask = x[1]  # 对padding部分进行mask，和句子一个size，padding部分用0表示，如：[1, 1, 1, 1, 0, 0]
        _, pooled = self.bert(context)
        out = self.fc(pooled)
        return out


In [29]:
import pandas as pd
dataset = pd.read_csv('F:/weibo_hot/data/train_content.csv')

In [67]:
tokenizer = BertTokenizer.from_pretrained(bert_path)
a = Model()

In [54]:
input_ids = torch.tensor(tokenizer.encode('你是真的牛逼啊'))
input.size()

torch.Size([1, 7])

In [69]:
torch.nn.init.xavier_normal_(a.fc.weight)
output = a(input_ids.unsqueeze(0))
print(output)

tensor([[ 872, 3221, 4696, 4638, 4281, 6873, 1557]])
tensor([[-1.7071, -0.1923,  0.0906]], grad_fn=<AddmmBackward>)


In [253]:
import tqdm
from torch.utils.data import Dataset, DataLoader

class dataset(Dataset):
    def __init__(self, data='train'):
        super(dataset,self).__init__()
        self.set = data
        
    def load_csv(self, path):
        self.df = pd.read_csv(path+'/'+self.set+'_content.csv')
        self.tokenizer = BertTokenizer.from_pretrained(bert_path)
        self.df.loc[:,'content'] = self.df.content.apply(str).apply(self.tokenizer.encode)
        self.max_length = max(map(len,self.df.content))+1
        self.size = len(self.df)
    def __getitem__(self, index):
        sample = self.df.iloc[index]
        x = sample.loc['content']
        x = self.tokenizer.encode('[CLS]') + x
        mask = []
        seq_len = len(x)
        print(seq_len)
        if seq_len < self.max_length:
            mask = [1] * seq_len + [0] * (self.max_length - seq_len)
            x += ([0] * (self.max_length - seq_len))
        else:
            mask = [1] * self.max_length
            x = x[:self.max_length]
            seq_len = self.max_length
    
        return {'x':torch.LongTensor(x),
                'y_c':torch.LongTensor([sample.comment_count]),
                'y_f':torch.LongTensor([sample.forward_count]),
                'y_l':torch.LongTensor([sample.like_count]),
                'len':torch.LongTensor([seq_len]),
                'mask':torch.LongTensor(mask)
        }
    def __len__(self):
        return self.size

In [254]:
datas = dataset(data='val')
datas.load_csv(data_path)

In [255]:
datas.max_length

211

In [256]:
datas[0]

141


{'x': tensor([ 101, 3295, 2190, 4636, 2428,  756, 5596, 6380,  756,  677,  837, 3152,
          816, 3683, 6772, 1962, 1936, 8024, 1920, 3152,  816, 1963,  862, 4746,
         7313,  677,  837, 4638, 8043,  809, 1184, 4339, 3844, 3221, 3302, 1218,
         1690,  924, 2100, 3680,  702, 3152,  816, 4638, 4294, 2519, 4772, 8024,
         4500, 2787, 4999, 1044, 5815, 1357, 2521,  677,  837, 3152,  816, 4638,
         4294, 2519, 4772, 8024, 5735, 1762, 3302, 1218, 1690, 2823, 1168, 4685,
         1398, 4638, 8024, 2218, 4684, 2970, 6819, 1726,  677,  837, 2768, 1216,
          511, 3219, 1921, 6435, 3136, 4636, 2428, 1040, 2475, 8024, 2533, 4761,
         3221, 9745, 8197,  711, 4294, 2519, 4772, 3297, 2207, 1296,  855, 8024,
         5735, 3378, 3152,  816,  680, 3302, 1218, 1690, 3152,  816, 1372, 3300,
         1126, 1282,  153, 2345, 2460, 8024, 6929,  720, 1372,  677,  837, 1259,
         1419, 2345, 2460, 4638, 9745, 8197, 2218, 3616,  749,    0,    0,    0,
            0,    0,   

In [257]:
torch.tensor(tokenizer.encode('[CLS]'))

tensor([101])

In [258]:
len(datas)

245924

In [259]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cuda"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=datas, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [262]:
batch = generate_batches(dataset,10,shuffle=True,
                     drop_last=True, device="cuda")

In [263]:
for batch_index, batch_dict in enumerate(batch):
    print

4
7
3
95
21
6
7
93
17
137


TypeError: 'Tensor' object is not callable

In [ ]:
import numpy as np


In [ ]:
a = torch.tensor(np.zeros(5))

In [ ]:
a.to('cuda')

In [ ]:
a